In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!nvidia-smi

In [ ]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import TrainingArguments, Trainer

## 🤗 Hugging Face Datasets Kütüphanesi

### Parametreler

- **`path`**:
  - Yüklenecek veri setinin yolu veya ismi. Örnek: "imdb", "glue".

- **`name`**:
  - Yüklenecek veri setinin alt kümesi. Örnek: "sst2" (GLUE için).

- **`data_dir`**:
  - Veri dosyalarının bulunduğu dizin.

- **`data_files`**:
  - Yüklenecek veri dosyaları.

- **`split`**:
  - Veri setinin bölünmesi (örneğin "train", "test").

- **`cache_dir`**:
  - Verinin önbelleğe alınacağı dizin.

- **`features`**:
  - Özelliklerin açıkça belirtildiği yer.

- **`download_config`**:
  - İndirme yapılandırma ayarları.

- **`download_mode`**:
  - İndirme modu: "reuse_dataset_if_exists", "reuse_cache_if_exists", "force_redownload".

- **`verification_mode`**:
  - Veri setinin doğrulama modu.

- **`ignore_verifications`**:
  - Artık kullanılmıyor, doğrulamaları atlamak için.

- **`keep_in_memory`**:
  - Eğer True ise, veri seti bellekte tutulur.

- **`save_infos`**:
  - Eğer True ise, veri seti bilgileri kaydedilir.

- **`revision`**:
  - Yüklenecek veri setinin versiyonu veya commit ID'si.

- **`token`**:
  - Private veri setleri için kullanılır, bir token sağlar.

- **`use_auth_token`**:
  - Artık kullanılmıyor, oturum açma token'ı için.

- **`task`**:
  - Artık kullanılmıyor, veri seti yükleme görevini belirtmek için.

- **`streaming`**:
  - Eğer True ise, veri seti akış modunda yüklenir.

- **`num_proc`**:
  - Çok işlemcili veri işleme için işlemci sayısı.

- **`storage_options`**:
  - Depolama seçenekleri.

- **`trust_remote_code`**:
  - Eğer True ise, uzaktan kod çalıştırmaya izin verir.

- **`**config_kwargs`**:
  - Diğer ek yapılandırma argümanları.


In [ ]:
dataset = load_dataset('Trelis/tiny-shakespeare', split='train')

dataset

## 🤗 Hugging Face Tokenizer Kütüphanesi

### Parametreler

- **`text`**:
  - Tokenize edilecek metin veya metinlerin listesi.

- **`text_pair`**:
  - İkinci bir metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`text_target`**:
  - Hedef metin veya metinlerin listesi (genellikle seq2seq modelleri için kullanılır).

- **`text_pair_target`**:
  - Hedef ikinci metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`add_special_tokens`**:
  - Özel tokenler ekler (örneğin [CLS], [SEP]). True olması, modeli daha iyi performans gösterir.

- **`padding`**:
  - Padding stratejisi: True, False, "longest", "max_length".

- **`truncation`**:
  - Kesme stratejisi: True, False, "longest_first", "only_first".

- **`max_length`**:
  - Maksimum token sayısı. Yüksek değerler daha fazla bilgi taşır ama daha fazla bellek kullanır.

- **`stride`**:
  - Kesme sırasında kayma boyutu. Uzun metinler için daha küçük değerler kullanışlı olabilir.

- **`is_split_into_words`**:
  - Eğer True ise, metin kelimelere bölünmüş olarak kabul edilir.

- **`pad_to_multiple_of`**:
  - Padding boyutunun bir katı olacak şekilde padding ekler.

- **`return_tensors`**:
  - Döndürülecek tensör tipi: 'pt', 'tf', 'np'.

- **`return_token_type_ids`**:
  - Eğer True ise, token tipi ID'lerini döndürür.

- **`return_attention_mask`**:
  - Eğer True ise, attention maskelerini döndürür.

- **`return_overflowing_tokens`**:
  - Eğer True ise, taşan tokenleri döndürür.

- **`return_special_tokens_mask`**:
  - Eğer True ise, özel token maskesini döndürür.

- **`return_offsets_mapping`**:
  - Eğer True ise, offset mapping döndürür.

- **`return_length`**:
  - Eğer True ise, token uzunluklarını döndürür.

- **`verbose`**:
  - Eğer True ise, işlem hakkında daha fazla bilgi verir.

- **`**kwargs`**:
  - Diğer ek argümanlar.


In [ ]:
base_model_name = "openai-community/gpt2"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def text_tokenizer(text):
    return tokenizer(
        text['text'], 
        max_length=128, 
        truncation=True, 
        padding='max_length', 
        return_tensors='pt'
    )

train_data = dataset['train'].map(text_tokenizer, batched=True)
test_data = dataset['test'].map(text_tokenizer, batched=True)
val_data = dataset['validation'].map(text_tokenizer, batched=True)

train_data

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model_name)

## 🤗 Hugging Face TrainingArguments Kütüphanesi

### Parametreler

- **`output_dir`**:
  - Eğitim sırasında ve sonrasında sonuçların kaydedileceği dizin.

- **`overwrite_output_dir`**:
  - Eğer True ise, mevcut `output_dir` içeriğini üzerine yazar.

- **`do_train`**:
  - Eğitim işleminin yapılıp yapılmayacağını belirtir.

- **`do_eval`**:
  - Değerlendirme işleminin yapılıp yapılmayacağını belirtir.

- **`do_predict`**:
  - Tahmin işleminin yapılıp yapılmayacağını belirtir.

- **`eval_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`prediction_loss_only`**:
  - Eğer True ise, sadece kayıp hesaplanır, tahminler döndürülmez.

- **`per_device_train_batch_size`**:
  - Her bir cihaz (GPU/CPU) için eğitim batch boyutu.

- **`per_device_eval_batch_size`**:
  - Her bir cihaz (GPU/CPU) için değerlendirme batch boyutu.

- **`gradient_accumulation_steps`**:
  - Gradient biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama eğitim süresini uzatır.

- **`eval_accumulation_steps`**:
  - Değerlendirme biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama değerlendirme süresini uzatır.

- **`eval_delay`**:
  - Eğitim başladıktan sonra ilk değerlendirmenin yapılacağı adım sayısı.

- **`learning_rate`**:
  - Öğrenme oranı. Yüksek değerler daha hızlı öğrenir ama aşırı öğrenmeye (overfitting) yol açabilir.

- **`weight_decay`**:
  - Ağırlıkların küçülme oranı. Yüksek değerler modelin genelleme yeteneğini artırabilir ama öğrenme yavaşlar.

- **`adam_beta1`**:
  - Adam optimizasyon algoritması için beta1 parametresi.

- **`adam_beta2`**:
  - Adam optimizasyon algoritması için beta2 parametresi.

- **`adam_epsilon`**:
  - Adam optimizasyon algoritması için epsilon parametresi.

- **`max_grad_norm`**:
  - Gradientlerin maksimum normu. Büyük değerler modelin stabilitesini artırabilir.

- **`num_train_epochs`**:
  - Eğitim için epoch sayısı. Yüksek değerler daha fazla öğrenme sağlar ama aşırı öğrenmeye yol açabilir.

- **`max_steps`**:
  - Maksimum eğitim adım sayısı. -1 ise, tüm epoch'lar tamamlanır.

- **`lr_scheduler_type`**:
  - Öğrenme oranı zamanlayıcı tipi.

- **`lr_scheduler_kwargs`**:
  - Öğrenme oranı zamanlayıcısı için ek parametreler.

- **`warmup_ratio`**:
  - Öğrenme oranı ısınma oranı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`warmup_steps`**:
  - Öğrenme oranı ısınma adım sayısı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`log_level`**:
  - Log seviyesi. 'passive', 'info', 'warning', 'error' veya 'critical'.

- **`log_level_replica`**:
  - Çoklu GPU eğitiminde log seviyesi.

- **`log_on_each_node`**:
  - Çoklu node eğitiminde her node için loglama yapılır.

- **`logging_dir`**:
  - TensorBoard logları için dizin.

- **`logging_strategy`**:
  - Loglama stratejisi: 'no', 'steps' veya 'epoch'.

- **`logging_first_step`**:
  - Eğer True ise, ilk adımda loglama yapılır.

- **`logging_steps`**:
  - Kaç adımda bir loglama yapılacağı.

- **`logging_nan_inf_filter`**:
  - Eğer True ise, NaN ve sonsuz değerler loglanmaz.

- **`save_strategy`**:
  - Modelin kaydedilme stratejisi: 'no', 'steps' veya 'epoch'.

- **`save_steps`**:
  - Kaç adımda bir modelin kaydedileceği.

- **`save_total_limit`**:
  - Maksimum kaç model kaydedileceği.

- **`save_safetensors`**:
  - Modelin güvenli tensör formatında kaydedilip kaydedilmeyeceği.

- **`save_on_each_node`**:
  - Eğer True ise, her node'da model kaydedilir.

- **`save_only_model`**:
  - Eğer True ise, sadece model kaydedilir, optimizer ve lr scheduler kaydedilmez.

- **`restore_callback_states_from_checkpoint`**:
  - Callback durumlarının checkpoint'ten geri yüklenip yüklenmeyeceği.

- **`no_cuda`**:
  - Eğer True ise, GPU kullanılmaz.

- **`use_cpu`**:
  - Eğer True ise, CPU kullanılır.

- **`use_mps_device`**:
  - Eğer True ise, MacOS Metal Performance Shaders kullanılır.

- **`seed`**:
  - Rastgelelik için seed değeri.

- **`data_seed`**:
  - Veri yükleme için seed değeri.

- **`jit_mode_eval`**:
  - Eğer True ise, PyTorch JIT modu kullanılır.

- **`use_ipex`**:
  - Eğer True ise, Intel Extension for PyTorch kullanılır.

- **`bf16`**:
  - Eğer True ise, bfloat16 kullanılır.

- **`fp16`**:
  - Eğer True ise, float16 kullanılır.

- **`fp16_opt_level`**:
  - float16 optimizasyon seviyesi.

- **`half_precision_backend`**:
  - Yarı hassasiyet arka ucu: 'auto', 'amp' veya 'apex'.

- **`bf16_full_eval`**:
  - Eğer True ise, bfloat16 tam değerlendirme yapılır.

- **`fp16_full_eval`**:
  - Eğer True ise, float16 tam değerlendirme yapılır.

- **`tf32`**:
  - Eğer True ise, TensorFloat-32 kullanılır.

- **`local_rank`**:
  - Çoklu GPU eğitiminde lokal rank.

- **`ddp_backend`**:
  - DDP backend tipi: 'nccl', 'gloo', 'mpi'.

- **`tpu_num_cores`**:
  - TPU çekirdek sayısı.

- **`tpu_metrics_debug`**:
  - TPU metriklerinin debug bilgilerini içerir.

- **`debug`**:
  - Debug seçeneği: 'underflow_overflow', 'poor_optimization'.

- **`dataloader_drop_last`**:
  - Eğer True ise, dataloader son batch'i düşürür.

- **`eval_steps`**:
  - Değerlendirme adım sayısı.

- **`dataloader_num_workers`**:
  - Dataloader için çalışan sayısı. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`dataloader_prefetch_factor`**:
  - Dataloader için prefetch faktörü. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`past_index`**:
  - Geçmiş index.

- **`run_name`**:
  - Çalışma ismi.

- **`disable_tqdm`**:
  - Eğer True ise, tqdm progress bar devre dışı bırakılır.

- **`remove_unused_columns`**:
  - Eğer True ise, kullanılmayan sütunlar veri setinden kaldırılır.

- **`label_names`**:
  - Label isimleri.

- **`load_best_model_at_end`**:
  - Eğitim sonunda en iyi modelin yüklenip yüklenmeyeceği.

- **`metric_for_best_model`**:
  - En iyi modelin seçilmesi için kullanılacak metrik.

- **`greater_is_better`**:
  - Eğer True ise, metriklerde yüksek değerler daha iyi olarak kabul edilir.

- **`ignore_data_skip`**:
  - Eğer True ise, veri atlamaları göz ardı edilir.

- **`fsdp`**:
  - Fully Sharded Data Parallel ayarları.

- **`fsdp_min_num_params`**:
  - Fully Sharded Data Parallel için minimum parametre sayısı.

- **`fsdp_config`**:
  - Fully Sharded Data Parallel için ek ayarlar.

- **`fsdp_transformer_layer_cls_to_wrap`**:
  - Fully Sharded Data Parallel için transformer layer sınıfı.

- **`accelerator_config`**:
  - Accelerator için ek ayarlar.

- **`deepspeed`**:
  - DeepSpeed yapılandırması.

- **`label_smoothing_factor`**:
  - Label smoothing faktörü.

- **`optim`**:
  - Optimizasyon algoritması.

- **`optim_args`**:
  - Optimizasyon algoritması için ek argümanlar.

- **`adafactor`**:
  - Eğer True ise, Adafactor optimizasyon algoritması kullanılır.

- **`group_by_length`**:
  - Eğer True ise, input uzunluğuna göre gruplanır.

- **`length_column_name`**:
  - Uzunluk sütunu ismi.

- **`report_to`**:
  - Raporlama platformları (örneğin 'wandb').

- **`ddp_find_unused_parameters`**:
  - DDP'de kullanılmayan parametrelerin bulunması.

- **`ddp_bucket_cap_mb`**:
  - DDP bucket kapasitesi.

- **`ddp_broadcast_buffers`**:
  - DDP broadcast buffer'ları.

- **`dataloader_pin_memory`**:
  - Dataloader için bellek pinleme.

- **`dataloader_persistent_workers`**:
  - Dataloader için kalıcı çalışanlar.

- **`skip_memory_metrics`**:
  - Eğer True ise, bellek metrikleri atlanır.

- **`use_legacy_prediction_loop`**:
  - Eğer True ise, eski tahmin döngüsü kullanılır.

- **`push_to_hub`**:
  - Eğer True ise, model Hugging Face Hub'a itilir.

- **`resume_from_checkpoint`**:
  - Checkpoint'ten devam edilir.

- **`hub_model_id`**:
  - Hugging Face Hub model ID.

- **`hub_strategy`**:
  - Hub stratejisi.

- **`hub_token`**:
  - Hugging Face Hub token.

- **`hub_private_repo`**:
  - Eğer True ise, özel repo kullanılır.

- **`hub_always_push`**:
  - Eğer True ise, her zaman Hub'a itilir.

- **`gradient_checkpointing`**:
  - Eğer True ise, gradient checkpointing yapılır.

- **`gradient_checkpointing_kwargs`**:
  - Gradient checkpointing için ek argümanlar.

- **`include_inputs_for_metrics`**:
  - Eğer True ise, değerlendirme metrikleri için inputlar da dahil edilir.

- **`eval_do_concat_batches`**:
  - Eğer True ise, değerlendirme batch'leri birleştirilir.

- **`fp16_backend`**:
  - float16 arka ucu.

- **`evaluation_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`push_to_hub_model_id`**:
  - Hub model ID.

- **`push_to_hub_organization`**:
  - Hub organizasyon ID.

- **`push_to_hub_token`**:
  - Hub token.

- **`mp_parameters`**:
  - Model paralel parametreler.

- **`auto_find_batch_size`**:
  - Eğer True ise, batch boyutu otomatik bulunur.

- **`full_determinism`**:
  - Eğer True ise, deterministik eğitim yapılır.

- **`torchdynamo`**:
  - TorchDynamo kullanımı.

- **`ray_scope`**:
  - Ray scope.

- **`ddp_timeout`**:
  - DDP timeout süresi.

- **`torch_compile`**:
  - Eğer True ise, Torch compile kullanılır.

- **`torch_compile_backend`**:
  - Torch compile backend.

- **`torch_compile_mode`**:
  - Torch compile modu.

- **`dispatch_batches`**:
  - Batch dispatching.

- **`split_batches`**:
  - Batch splitting.

- **`include_tokens_per_second`**:
  - Eğer True ise, saniye başına token sayısı dahil edilir.

- **`include_num_input_tokens_seen`**:
  - Eğer True ise, görülen input token sayısı dahil edilir.

- **`neftune_noise_alpha`**:
  - Neptune noise alpha.

- **`optim_target_modules`**:
  - Optimizasyon hedef modüller.

- **`batch_eval_metrics`**:
  - Eğer True ise, batch değerlendirme metrikleri hesaplanır.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    hub_model_id='gpt2-wikitext-text-generation'
)

## 🤗 Hugging Face Trainer Kütüphanesi

### Parametreler

- **`model`**:
  - Eğitim ve değerlendirme için kullanılacak model.

- **`args`**:
  - Eğitim argümanları, `TrainingArguments` sınıfı ile oluşturulur.

- **`data_collator`**:
  - Veriyi toplamak için kullanılan data collator.

- **`train_dataset`**:
  - Eğitim için kullanılacak veri seti.

- **`eval_dataset`**:
  - Değerlendirme için kullanılacak veri seti.

- **`tokenizer`**:
  - Metin verilerini modelin anlayabileceği sayısal değerlere dönüştürür.

- **`model_init`**:
  - Modelin başlatılması için kullanılacak fonksiyon.

- **`compute_metrics`**:
  - Değerlendirme metriklerini hesaplamak için kullanılacak fonksiyon.

- **`callbacks`**:
  - Eğitim sürecini izlemek ve kontrol etmek için kullanılacak callback listesi.

- **`optimizers`**:
  - Optimizasyon algoritması ve öğrenme oranı zamanlayıcısı.

- **`preprocess_logits_for_metrics`**:
  - Metrik hesaplama için logits'i ön işleme fonksiyonu.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [ ]:
# ac4525a27cdcb34c068674c5fed00841eb0d9f4c

In [ ]:
trainer.train()

In [ ]:
from huggingface_hub import notebook_login

# Login to Hugging Face account
notebook_login()

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load dataset
dataset = load_dataset('Trelis/tiny-shakespeare', split='train')

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to('cuda:0')
# Set pad token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize function
def tokenize_function(examples):
    inputs = tokenizer(examples['Text'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    do_eval=False,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    push_to_hub=True,
    hub_model_id='gpt2-wikitext-text-generation',
    hub_strategy="end",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Train model
trainer.train()

# Move model to device
model = trainer.model
model.to('cuda:0')

# Input texts
input_texts = [
    "To be, or not to be, that is the question:",
    "All the world's a stage, and all the men and women merely players:"
]

# Generate text
text = input_texts[0]
inputs = tokenizer(text, return_tensors='pt').to(device)

prompt_text = "Once upon a time"
input_ids = tokenizer.encode(prompt_text, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    early_stopping=True
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

2024-06-19 18:05:43.283789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 18:05:43.283920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 18:05:43.416533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/472 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/49 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
